# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow opencv-python mediapipe sklearn matplotlib

# 2. Keypoints using MP Holistic

In [2]:
import cv2
import os
import copy
import argparse
import itertools
from collections import Counter
from collections import deque
from matplotlib import pyplot as plt
import time
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scipy import stats
import tensorflow as tf

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    #results = model.process(image)                 # Make prediction
    results = hands.process(image)
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv2.boundingRect(landmark_array)

    return [x, y, x + w, y + h]


def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y, landmark_z]) 
            
    return landmark_point


def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y, base_z = 0, 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y, base_z = landmark_point[0], landmark_point[1], landmark_point[2]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y
        temp_landmark_list[index][2] = temp_landmark_list[index][2] - base_z

    # Convert to a one-dimensional list
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list)) 

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))
    
    if len(temp_landmark_list)==126: 
        temp_landmark_list
    elif len(temp_landmark_list)==63:
        temp_landmark_list.extend(np.zeros(21*3))
    else:
        temp_landmark_list.extend(np.zeros(21*6))
        
    return temp_landmark_list

def draw_bounding_rect(use_brect, image, brect):
    if use_brect:
        # Outer rectangle
        cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                     (0, 0, 0), 1)

    return image

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
use_brect = True

# Set mediapipe model 
with mp_hands.Hands(static_image_mode=False, model_complexity=1, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
            
        # Read feed
        success, image = cap.read()
        # Rescale the frame by a factor of 0.5
        image = cv2.resize(image, None, fx=0.5, fy=0.5)
        debug_image = copy.deepcopy(image)
        if not success:
              print("Ignoring empty camera frame.")
              # If loading a video, use 'break' instead of 'continue'.
              continue
                
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        
        handedness = results.multi_handedness
        print(results)
        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
              for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                               mp_drawing_styles.get_default_hand_landmarks_style(), 
                                               mp_drawing_styles.get_default_hand_connections_style())
                    # Bounding box calculation
                    brect = calc_bounding_rect(debug_image, hand_landmarks)
                    
                    # Landmark calculation
                    landmark_list = calc_landmark_list(debug_image, hand_landmarks)

                    # Conversion to relative coordinates / normalized coordinates
                    pre_processed_landmark_list = pre_process_landmark(landmark_list)                    
                    
                    # Drawing part
                    image = draw_bounding_rect(use_brect, image, brect)   
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [27]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

NameError: name 'frame' is not defined

In [32]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

In [69]:
# To draw landmark For static images:
IMAGE_FILES = []
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print handedness and draw hand landmarks on the image.
    print('Handedness:', results.multi_handedness)
    if not results.multi_hand_landmarks:
      continue
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()
    for hand_landmarks in results.multi_hand_landmarks:
      print('hand_landmarks:', hand_landmarks)
      print(
          f'Index finger tip coordinates: (',
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
      )
      mp_drawing.draw_landmarks(
          annotated_image,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          mp_drawing_styles.get_default_hand_landmarks_style(),
          mp_drawing_styles.get_default_hand_connections_style())
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
    # Draw hand world landmarks.
    if not results.multi_hand_world_landmarks:
      continue
    for hand_world_landmarks in results.multi_hand_world_landmarks:
      mp_drawing.plot_landmarks(
        hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

In [26]:
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

NameError: name 'image' is not defined

# 3. Extract Keypoint Values

In [71]:
result_test = pre_process_landmark(landmark_list)

In [72]:
result_test

[0.0,
 0.0,
 0.0,
 0.2,
 -0.08571428571428572,
 4.1558320820556106e-05,
 0.5,
 -0.05714285714285714,
 5.087956224260535e-06,
 0.7142857142857143,
 0.1,
 -4.4999973395880894e-05,
 0.8285714285714286,
 0.15714285714285714,
 -9.211256895191866e-05,
 0.6285714285714286,
 -0.14285714285714285,
 -0.00013188801234831666,
 1.0,
 0.07142857142857142,
 -0.0001976817080748816,
 1.0,
 0.15714285714285714,
 -0.0002013367500420011,
 0.9142857142857143,
 0.15714285714285714,
 -0.0001899366565844726,
 0.5857142857142857,
 -0.04285714285714286,
 -0.0001881264848100171,
 0.9571428571428572,
 0.15714285714285714,
 -0.0002143766364612633,
 0.9142857142857143,
 0.15714285714285714,
 -0.00017518575763573503,
 0.8428571428571429,
 0.15714285714285714,
 -0.0001545726221318639,
 0.5,
 0.1,
 -0.00023523228836356427,
 0.8285714285714286,
 0.15714285714285714,
 -0.00025225630623399044,
 0.7857142857142857,
 0.15714285714285714,
 -0.00019530754065384722,
 0.7142857142857143,
 0.15714285714285714,
 -0.0001656215271

In [73]:
len(result_test)

126

In [74]:
np.save('0', result_test)

In [75]:
np.load('0.npy')

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  2.00000000e-01,
       -8.57142857e-02,  4.15583208e-05,  5.00000000e-01, -5.71428571e-02,
        5.08795622e-06,  7.14285714e-01,  1.00000000e-01, -4.49999734e-05,
        8.28571429e-01,  1.57142857e-01, -9.21125690e-05,  6.28571429e-01,
       -1.42857143e-01, -1.31888012e-04,  1.00000000e+00,  7.14285714e-02,
       -1.97681708e-04,  1.00000000e+00,  1.57142857e-01, -2.01336750e-04,
        9.14285714e-01,  1.57142857e-01, -1.89936657e-04,  5.85714286e-01,
       -4.28571429e-02, -1.88126485e-04,  9.57142857e-01,  1.57142857e-01,
       -2.14376636e-04,  9.14285714e-01,  1.57142857e-01, -1.75185758e-04,
        8.42857143e-01,  1.57142857e-01, -1.54572622e-04,  5.00000000e-01,
        1.00000000e-01, -2.35232288e-04,  8.28571429e-01,  1.57142857e-01,
       -2.52256306e-04,  7.85714286e-01,  1.57142857e-01, -1.95307541e-04,
        7.14285714e-01,  1.57142857e-01, -1.65621527e-04,  3.85714286e-01,
        1.57142857e-01, -

# 4. Setup Folders for Collection

In [76]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Lock_person','Unlock_person','Go_to_base','Follow', 'Stop', 'Turn_left', 'Turn_right', 'Move_forward', 'Move_backward'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [20]:
for action in actions: 
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [35]:
import cv2
import os
import copy
import argparse
import itertools
from collections import Counter
from collections import deque
from matplotlib import pyplot as plt
import time
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scipy import stats
import tensorflow as tf

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    #results = model.process(image)                 # Make prediction
    results = hands.process(image)
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv2.boundingRect(landmark_array)

    return [x, y, x + w, y + h]


def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y, landmark_z]) 
            
    return landmark_point


def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y, base_z = 0, 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y, base_z = landmark_point[0], landmark_point[1], landmark_point[2]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y
        temp_landmark_list[index][2] = temp_landmark_list[index][2] - base_z

    # Convert to a one-dimensional list
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list)) 

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list)) 
    
    if len(temp_landmark_list)==126: 
        temp_landmark_list
    elif len(temp_landmark_list)==63:
        temp_landmark_list.extend(np.zeros(21*3))
    else:
        temp_landmark_list.extend(np.zeros(21*6))
    
    return temp_landmark_list


def draw_bounding_rect(use_brect, image, brect):
    if use_brect:
        # Outer rectangle
        cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                     (0, 0, 0), 1)

    return image

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
use_brect = True

# Set mediapipe model 
with mp_hands.Hands(model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(1,no_sequences+1):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):        
                # Read feed
                success, image = cap.read()
                debug_image = copy.deepcopy(image)
                if not success:
                      print("Ignoring empty camera frame.")
                      # If loading a video, use 'break' instead of 'continue'.
                      continue

                # To improve performance, optionally mark the image as not writeable to
                # pass by reference.
                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = hands.process(image)

                handedness = results.multi_handedness
                print(results)
                # Draw the hand annotations on the image.
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                if results.multi_hand_landmarks:
                      for hand_landmarks in results.multi_hand_landmarks:
                            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                                       mp_drawing_styles.get_default_hand_landmarks_style(), 
                                                       mp_drawing_styles.get_default_hand_connections_style())
                            # Bounding box calculation
                            brect = calc_bounding_rect(debug_image, hand_landmarks)

                            # Landmark calculation
                            landmark_list = calc_landmark_list(debug_image, hand_landmarks)

                            # Conversion to relative coordinates / normalized coordinates
                            pre_processed_landmark_list = pre_process_landmark(landmark_list)

                            # Drawing part
                            image = draw_bounding_rect(use_brect, image, brect)   

                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 6, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 4, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000) # defines the wait time between collection
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 4, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)

                # NEW Export keypoints
                keypoints = pre_process_landmark(landmark_list)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [34]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [77]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [78]:
label_map = {label:num for num, label in enumerate(actions)}

In [79]:
label_map

{'Lock_person': 0,
 'Unlock_person': 1,
 'Go_to_base': 2,
 'Follow': 3,
 'Stop': 4,
 'Turn_left': 5,
 'Turn_right': 6,
 'Move_forward': 7,
 'Move_backward': 8}

In [86]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [87]:
np.array(sequences).shape

(270, 30, 231)

In [88]:
np.array(labels).shape

(270,)

In [89]:
X = np.array(sequences)

In [90]:
X.shape

(270, 30, 231)

In [91]:
y = to_categorical(labels).astype(int)

In [92]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [94]:
y_test.shape

(14, 9)

# 7. Build and Train LSTM Neural Network

In [99]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [100]:
log_dir = os.path.join('Logs')
#tb_callback = TensorBoard(log_dir=log_dir)
tb_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta=0.001,
    #patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=300
)

In [101]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(30,231)))
model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [103]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [104]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
8/8 [==============================] - 3s 117ms/step - loss: 2.0111 - categorical_accuracy: 0.4180
Epoch 2/2000
8/8 [==============================] - 1s 116ms/step - loss: 4.2868 - categorical_accuracy: 0.4609
Epoch 3/2000
8/8 [==============================] - 1s 117ms/step - loss: 1.1690 - categorical_accuracy: 0.5781
Epoch 4/2000
8/8 [==============================] - 1s 116ms/step - loss: 1.3897 - categorical_accuracy: 0.7070
Epoch 5/2000
8/8 [==============================] - 1s 115ms/step - loss: 1.3428 - categorical_accuracy: 0.7695
Epoch 6/2000
8/8 [==============================] - 1s 118ms/step - loss: 1.0940 - categorical_accuracy: 0.7578
Epoch 7/2000
8/8 [==============================] - 1s 117ms/step - loss: 0.7056 - categorical_accuracy: 0.7500
Epoch 8/2000
8/8 [==============================] - 1s 117ms/step - loss: 0.7646 - categorical_accuracy: 0.7695
Epoch 9/2000
8/8 [==============================] - 1s 117ms/step - loss: 0.8741 - categorical_accuracy:

In [105]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_36 (LSTM)              (None, 30, 128)           184320    
                                                                 
 lstm_37 (LSTM)              (None, 30, 256)           394240    
                                                                 
 lstm_38 (LSTM)              (None, 128)               197120    
                                                                 
 dense_36 (Dense)            (None, 128)               16512     
                                                                 
 dense_37 (Dense)            (None, 64)                8256      
                                                                 
 dense_38 (Dense)            (None, 9)                 585       
                                                                 
Total params: 801,033
Trainable params: 801,033
Non-t

# 8. Make Predictions

In [106]:
res = model.predict(X_test)

1/1 [==============================] - 0s 284ms/step


In [107]:
len(X_test)

14

In [108]:
actions[np.argmax(res[4])]

'Turn_right'

In [109]:
np.argmax(res[4])

6

In [110]:
actions[np.argmax(y_test[4])]

'Turn_right'

# 9. Save Weights

In [111]:
model.save('action.h5')

In [217]:
del model

In [112]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [113]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [114]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 40ms/step


In [115]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [116]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[13,  0],
        [ 0,  1]],

       [[11,  0],
        [ 0,  3]],

       [[13,  0],
        [ 0,  1]],

       [[12,  0],
        [ 0,  2]],

       [[10,  0],
        [ 0,  4]],

       [[12,  0],
        [ 0,  2]],

       [[13,  0],
        [ 0,  1]]], dtype=int64)

In [117]:
accuracy_score(ytrue, yhat)

1.0

# 11. Test in Real Time

In [66]:
from scipy import stats

In [119]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [68]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

TypeError: only size-1 arrays can be converted to Python scalars

<Figure size 1800x1800 with 0 Axes>

In [120]:
import cv2
import os
import copy
import argparse
import itertools
from collections import Counter
from collections import deque
from matplotlib import pyplot as plt
import time
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scipy import stats
import tensorflow as tf

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Lock_person','Unlock_person','Go_to_base','Follow', 'Stop', 'Turn_left', 'Turn_right', 'Move_forward', 'Move_backward'])

# Preprocessing data
label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(30,231)))
model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.load_weights('action.h5')

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        #cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        #cv2.rectangle(output_frame,(384,0),(510,128),(0,255,0),-1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

# 1. New detection variables

# Sequence variable will collect the 30 frames from the webcam via OpenCV
sequence = []

# Sentence will concatinate the history of gesture detected.
sentence = []

predictions = []

# Threshold will render the results which are above this threshold
threshold = 0.95

def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv2.boundingRect(landmark_array)

    return [x, y, x + w, y + h]


def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y, landmark_z]) 
            
    return landmark_point


def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list)) 

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list)) 
    
    if len(temp_landmark_list)==126: 
        temp_landmark_list
    elif len(temp_landmark_list)==63:
        temp_landmark_list.extend(np.zeros(21*3))
    else:
        temp_landmark_list.extend(np.zeros(21*6))
    
    return temp_landmark_list

def draw_bounding_rect(use_brect, image, brect):
    if use_brect:
        # Outer rectangle
        cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                     (100, 150, 50), 6)

    return image


# For webcam input:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
use_brect = True

with mp_hands.Hands(model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
      while cap.isOpened():
        success, image = cap.read()
        # Rescale the frame by a factor of 0.5
        image = cv2.resize(image, None, fx=0.5, fy=0.5)
        
        debug_image = copy.deepcopy(image)
        if not success:
              print("Ignoring empty camera frame.")
              # If loading a video, use 'break' instead of 'continue'.
              continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        handedness = results.multi_handedness
        print(results)
        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_hand_landmarks:
              for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                               mp_drawing_styles.get_default_hand_landmarks_style(), 
                                               mp_drawing_styles.get_default_hand_connections_style())
                    # Bounding box calculation
                    brect = calc_bounding_rect(debug_image, hand_landmarks)
                    
                    # Landmark calculation
                    landmark_list = calc_landmark_list(debug_image, hand_landmarks)

                    # Conversion to relative coordinates / normalized coordinates
                    pre_processed_landmark_list = pre_process_landmark(landmark_list)
                    
                    '''if len(pre_processed_landmark_list)==126: 
                        continue
                    elif len(pre_processed_landmark_list)==63:
                        pre_processed_landmark_list.extend(np.zeros(21*3))
                    else:
                        pre_processed_landmark_list.extend(np.zeros(21*6))'''
                    
                    # Drawing part
                    image = draw_bounding_rect(use_brect, image, brect)    
        
        # 2. Prediction logic
        sequence.append(pre_processed_landmark_list)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)

            
        cv2.rectangle(image, (0,0), (2000, 40), (200, 197, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', image) #cv2.flip(image, 1))
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

ValueError: in user code:

    File "C:\ProgramData\anaconda3\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\anaconda3\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\anaconda3\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "C:\ProgramData\anaconda3\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "C:\ProgramData\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Inp

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained LSTM model
model = load_model('your_model_path.h5')

# Map class indices to action labels
actions = ["Lock Person", "Unlock Person", "Go to Base", "Follow", "Stop", "Turn Left", "Turn Right"]

# Set up MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Webcam parameters
width, height = 640, 480

# Open webcam
cap = cv2.VideoCapture(0)
cap.set(3, width)
cap.set(4, height)

while True:
    # Read frame from webcam
    ret, frame = cap.read()

    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Holistic
    with mp_holistic.Holistic(static_image_mode=False, model_complexity=2, min_detection_confidence=0.5,
                              min_tracking_confidence=0.5) as holistic:
        # Perform landmark detection
        results = holistic.process(frame_rgb)

        # Check if landmarks are detected
        if results.pose_landmarks:

            # Extract relevant landmarks (arms and nose)
            arm_landmarks_indices = [mp_holistic.PoseLandmark.LEFT_WRIST, mp_holistic.PoseLandmark.RIGHT_WRIST,
                                     mp_holistic.PoseLandmark.LEFT_ELBOW, mp_holistic.PoseLandmark.RIGHT_ELBOW,
                                     mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_SHOULDER]

            landmark_coords = []
            for landmark_idx in arm_landmarks_indices:
                landmark = results.pose_landmarks.landmark[landmark_idx]
                landmark_coords.extend([landmark.x, landmark.y, landmark.z])

            # Reshape and normalize the landmark coordinates
            landmark_coords = np.array(landmark_coords).reshape(1, -1)
            landmark_coords = (landmark_coords - np.mean(landmark_coords)) / np.std(landmark_coords)

            # Make a prediction using the LSTM model
            prediction = model.predict(landmark_coords)
            action_label = actions[np.argmax(prediction)]
            action_probability = np.max(prediction)

            # Draw the predicted action label and probability on the frame
            cv2.putText(frame, f"{action_label} ({action_probability:.2f})", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 255, 0), 2, cv2.LINE_AA)

        # Draw landmarks on the frame
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))

    # Show the frame
    cv2.imshow('Dynamic Gesture Recognition', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


In [121]:
cap.release()
cv2.destroyAllWindows()